## Read labels json and print all unique ones

Ελέγξτε πόσες και ποιες κατηγορίες είναι διαθέσιμες στο υποσύνολο δεδομένων της άσκησης. Ορίστε νέο indexing για τις κατηγορίες

In [1]:
import json
from collections import Counter
import os
import pickle as pkl
import shutil
import random
import zarr
from torch.utils import data
import numpy as np

# labels of zarr files according to labels.json
f_labels = open(r"Exercise4\timematch_data\denmark\32VNH\2017\meta\labels.json")
labels_json_original = json.load(f_labels)

print(len(labels_json_original.keys()))
print(len(set(labels_json_original.values())))

5001
14


Οι διαθέσιμες  κατηγορίες στα δικά μας αρχικά δεδομένα είναι 14

In [2]:
unique_labels = set()
for val in labels_json_original.values():
    unique_labels.add(val)

labels_counter = Counter(labels_json_original.values())

for lab in unique_labels:
    print(lab, labels_counter[lab])

spring_barley 1141
winter_rye 317
spring_peas 17
winter_barley 352
horsebeans 28
spring_wheat 26
winter_wheat 856
winter_rapeseed 301
unknown 511
winter_triticale 42
corn 275
spring_triticale 2
spring_oat 120
meadow 1013


Τα δεδομένα κατηγοριών με λιγότερα από 200 δείγματα φεύγουν

In [3]:
def move_zarr_files_based_on_label(label_occurrences, zarr_folder_path, json_file_path, destination_folder_path):
    # Load the JSON file
    with open(json_file_path, 'r') as json_file:
        labels = json.load(json_file)
    
    # Ensure the destination folder exists
    os.makedirs(destination_folder_path, exist_ok=True)

    # Iterate over the zarr files in the given folder
    for file_name in os.listdir(zarr_folder_path):
        if file_name.endswith('.zarr'):
            file_number = file_name.replace('.zarr', '')
            
            # Check if the file number is in the labels dictionary
            if file_number in labels:
                label = labels[file_number]
                
                # Check if the occurrence of the label is less than 200
                if label_occurrences.get(label, 0) < 200:
                    source_path = os.path.join(zarr_folder_path, file_name)
                    destination_path = os.path.join(destination_folder_path, file_name)
                    
                    shutil.move(source_path, destination_path)
                    print(f"Moved {file_name} to {destination_folder_path}")

# move_zarr_files_based_on_label(
#     labels_counter,
#     "Exercise4/timematch_data/denmark/32VNH/2017/data",
#     r"C:\Users\Yannis\Downloads\edemm_earino\gewxwrika\proj3\Exercise4\timematch_data\denmark\32VNH\2017\meta\labels.json",
#     "moved_zarrs")

print(len(os.listdir("moved_zarrs")))

235


Φεύγουν και τα δείγματα άγνωστων κατηγοριών

In [4]:
def move_unknown_zarr(zarr_folder_path, json_file_path, destination_folder_path):
    with open(json_file_path, 'r') as json_file:
        labels = json.load(json_file)
    
    os.makedirs(destination_folder_path, exist_ok=True)

    # Iterate over the zarr files in the given folder
    for file_name in os.listdir(zarr_folder_path):
        if file_name.endswith('.zarr'):
            file_number = file_name.replace('.zarr', '')

            # Check if the file number is in the labels dictionary
            if file_number in labels:
                label = labels[file_number]

                if label == 'unknown':
                    source_path = os.path.join(zarr_folder_path, file_name)
                    destination_path = os.path.join(destination_folder_path, file_name)
                    
                    shutil.move(source_path, destination_path)
                    print(f"Moved {file_name} to {destination_folder_path}")

# move_unknown_zarr(
#     "Exercise4/timematch_data/denmark/32VNH/2017/data",
#     r"C:\Users\Yannis\Downloads\edemm_earino\gewxwrika\proj3\Exercise4\timematch_data\denmark\32VNH\2017\meta\labels.json",
#     "moved_unknowns")

print(len(os.listdir("moved_unknowns")))

511


Καινούριο dict από labels με τα δείγματα που έχουν απομείνει

In [5]:
# labels_200 will contain the labels with more than 200 occurrences
labels_200 = {}

for file_name in os.listdir("Exercise4/timematch_data/denmark/32VNH/2017/data"):
    if file_name.endswith('.zarr'):
        file_number = file_name.replace('.zarr', '')
        # print(file_number)
        
        # if the number of occurrences of label is greater than 200,
        # add it to the labels_200 dictionary
        # print(labels_counter[file_number])
        category = labels_json_original[file_number]
        if labels_counter[category] > 200:
            labels_200[file_number] = labels_json_original[file_number]
            # labels_200[int(file_number)] = labels_json_original[file_number]

# labels_200 = dict(sorted(labels_200.items()))
print(len(labels_200))

count = 0
for lab in labels_200:
    print(lab, labels_200[lab])
    count += 1
    if count == 7:
        break

labels_200_counter = Counter(labels_200.values())

print()

for lab in labels_200_counter:
    print(lab, labels_200_counter[lab])

4255
0 corn
1 corn
10 corn
100 winter_rapeseed
1000 spring_barley
1001 winter_rye
1002 spring_barley

corn 275
winter_rapeseed 301
spring_barley 1141
winter_rye 317
winter_barley 352
winter_wheat 856
meadow 1013


Αντιστοίχιση τελικών labels με αριθμητικά indices

In [6]:
class_to_idx = {cls: idx for idx, cls in enumerate(labels_200_counter)}

for key, val in class_to_idx.items():
    print(key, val)

corn 0
winter_rapeseed 1
spring_barley 2
winter_rye 3
winter_barley 4
winter_wheat 5
meadow 6


## Dataset Class

In [7]:
from torchvision.transforms import transforms
import torch

class RandomSamplePixels(object):
    """Randomly draw num_pixels from the available pixels in sample.
    If the total number of pixels is less than num_pixels, one arbitrary pixel is repeated.
    The valid_pixels keeps track of true and repeated pixels.

    Args:
        num_pixels (int): Number of pixels to sample.
    """

    def __init__(self, num_pixels):
        self.num_pixels = num_pixels

    def __call__(self, sample):
        
        pixels = sample['pixels']
        
        T, C, S = pixels.shape
        if S > self.num_pixels:
            indices = random.sample(range(S), self.num_pixels)
            x = pixels[:, :, indices]
            valid_pixels = np.ones(self.num_pixels)
        elif S < self.num_pixels:
            x = np.zeros((T, C, self.num_pixels))
            x[..., :S] = pixels
            x[..., S:] = np.stack([x[:, :, 0] for _ in range(S, self.num_pixels)], axis=-1)
            valid_pixels = np.array([1 for _ in range(S)] + [0 for _ in range(S, self.num_pixels)])
        else:
            x = pixels
            valid_pixels = np.ones(self.num_pixels)
        # Repeat valid_pixels across time
        valid_pixels = np.repeat(valid_pixels[np.newaxis].astype(np.float32), x.shape[0], axis=0)
        sample['pixels'] = x
        sample['valid_pixels'] = valid_pixels
        return sample

class Normalize(object):
    """Normalize by rescaling pixels to [0, 1]

    Args:
        max_pixel_value (int): Max value of pixels to move pixels to [0, 1]
    """

    def __init__(self, max_pixel_value=65535):
        self.max_pixel_value = max_pixel_value

        # approximate max values
        max_parcel_box_m = 10000
        max_perimeter = max_parcel_box_m * 4
        max_area = max_parcel_box_m ** 2
        max_perimeter_area_ratio = max_perimeter
        max_cover_ratio = 1.0
        self.max_extra_values = np.array([max_perimeter, max_area, max_perimeter_area_ratio, max_cover_ratio])

    def __call__(self, sample):
        sample['pixels'] = np.clip(sample['pixels'], 0, self.max_pixel_value).astype(np.float32) / self.max_pixel_value
        
        # if 'extra' in sample:
        #     sample['extra'] = sample['extra'].astype(np.float32) / self.max_extra_values
        
        return sample

class ToTensor(object):
    def __call__(self, sample):
        sample['pixels'] = torch.from_numpy(sample['pixels'].astype(np.float32))
        sample['valid_pixels'] = torch.from_numpy(sample['valid_pixels'].astype(np.float32))
        # sample['positions'] = torch.from_numpy(sample['positions'].astype(np.long))
        # if 'extra' in sample:
        #     sample['extra'] = torch.from_numpy(sample['extra'].astype(np.float32))
        if isinstance(sample['label'], int):
            sample['label'] = torch.tensor(sample['label']).long()
        return sample

In [8]:
# def ix(dct, n):
#     # get nth key of a dictionary
#     return list(dct)[n]
   
class PixelSetData(data.Dataset):
    def __init__(
            self,
            data_root,
            class_to_idx,
            true_labels,
            # num_pixels = 32,
            transform=None,
            ):

        self.folder = os.path.join(data_root)
        self.data_folder = os.path.join(self.folder, "data")
        # self.meta_folder = os.path.join(self.folder, "meta")
        self.data_labels = true_labels
        # self.num_pixels = num_pixels
        self.transform = transform
        self.class_to_idx = class_to_idx
        # self.idx_to_class = {v: k for k, v in self.class_to_idx.items()}

        self.samples = self._make_dataset()

        for i in range(10):
            print(self.samples[i])
    
    # def __getitem__(self, index):
    #     item = self.samples[index]
    #     zarr_arr = zarr.load(item[0])
    #     T, C, S = zarr_arr.shape
    #     # return 32 random pixels
    #     if S > self.num_pixels:
    #         indices = random.sample(range(S), self.num_pixels)
    #         x = zarr_arr[:, :, indices]
    #     elif S < self.num_pixels:
    #         x = np.zeros((T, C, self.num_pixels))
    #         x[..., :S] = zarr_arr
    #         x[..., S:] = np.stack([x[:, :, 0] for _ in range(S, self.num_pixels)], axis=-1)
    #     else:
    #         x = zarr_arr
    #     return x

    def __getitem__(self, index):
        path, parcel_idx, y = self.samples[index]
        pixels = zarr.load(path)  # (T, C, S)

        sample = {
            "index": index,
            "parcel_index": parcel_idx,  # mapping to metadata
            "pixels": pixels,
            "valid_pixels": np.ones(
                (pixels.shape[0], pixels.shape[-1]), dtype=np.float32),
            # "positions": np.array(self.date_positions),
            # "extra": np.array(extra),
            "label": y,
        }

        if self.transform is not None:
            sample = self.transform(sample)
        return sample
    
    def __len__(self):
        return len(self.samples)
    
    def _make_dataset(self):
        # metadata = pkl.load(open(os.path.join(self.meta_folder, "metadata.pkl"), "rb"))
        instances = []
        
        for zarr_file in os.listdir(self.data_folder):
            if zarr_file.endswith(".zarr"):

                zarr_idx = zarr_file.split(".")[-2]
                parcel_path = os.path.join(self.data_folder, f"{zarr_idx}.zarr")
                class_name = self.data_labels[zarr_idx]
                item = (parcel_path, zarr_idx, class_name)
                instances.append(item)
        
        return instances

In [9]:
train_transform = transforms.Compose([
        RandomSamplePixels(32),
        Normalize(),
        ToTensor(),
    ])

pixel_dataset = PixelSetData("Exercise4/timematch_data/denmark/32VNH/2017",
                             class_to_idx,
                             labels_200,
                             train_transform)
print()
print(len(pixel_dataset))

('Exercise4/timematch_data/denmark/32VNH/2017\\data\\0.zarr', '0', 'corn')
('Exercise4/timematch_data/denmark/32VNH/2017\\data\\1.zarr', '1', 'corn')
('Exercise4/timematch_data/denmark/32VNH/2017\\data\\10.zarr', '10', 'corn')
('Exercise4/timematch_data/denmark/32VNH/2017\\data\\100.zarr', '100', 'winter_rapeseed')
('Exercise4/timematch_data/denmark/32VNH/2017\\data\\1000.zarr', '1000', 'spring_barley')
('Exercise4/timematch_data/denmark/32VNH/2017\\data\\1001.zarr', '1001', 'winter_rye')
('Exercise4/timematch_data/denmark/32VNH/2017\\data\\1002.zarr', '1002', 'spring_barley')
('Exercise4/timematch_data/denmark/32VNH/2017\\data\\1003.zarr', '1003', 'winter_barley')
('Exercise4/timematch_data/denmark/32VNH/2017\\data\\1004.zarr', '1004', 'winter_rapeseed')
('Exercise4/timematch_data/denmark/32VNH/2017\\data\\1005.zarr', '1005', 'winter_rapeseed')

4255


In [18]:
ret = pixel_dataset[10]

for k, v in ret.items():
    if isinstance(v, torch.Tensor):
        print(k, type(v), v.shape)
    else:
        print(k, type(v), v)

index <class 'int'> 10
parcel_index <class 'str'> 1006
pixels <class 'torch.Tensor'> torch.Size([52, 10, 32])
valid_pixels <class 'torch.Tensor'> torch.Size([52, 32])
label <class 'str'> winter_wheat


In [20]:
ret = pixel_dataset[1901]

for k, v in ret.items():
    if isinstance(v, torch.Tensor):
        print(k, type(v), v.shape)
    else:
        print(k, type(v), v)

index <class 'int'> 1901
parcel_index <class 'str'> 2999
pixels <class 'torch.Tensor'> torch.Size([52, 10, 32])
valid_pixels <class 'torch.Tensor'> torch.Size([52, 32])
label <class 'str'> winter_wheat
